# Students Do: Credit Risk Classification with Amazon SageMaker

* **Dataset:** German Credit Risk Dataset - Prof. Dr. Hans Hofmann (original source: [ics.uci.edu](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)), download: [Kaggle](https://www.kaggle.com/uciml/german-credit))

* **Goal:** Classify the credit risk of a person as described by a given set of input features.

**Note:** You should import and run this notebook into your notebook intance on Amazon SageMaker.

In [ ]:
# Initial imports
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

%matplotlib inline

## Loading the data

In [ ]:
# Upload the german_credit_data.csv file (located in ../Resources/) through through JupyterLab
file_path = Path("Data/german_credit_data.csv")
df = pd.read_csv(file_path, index_col=0)
df.head()

In [ ]:
# Creating a DataFrame with the features (include all columns except "Risk")
features_df = df.drop("Risk", axis=1)
features_df.head()

In [ ]:
# Creating a DataFrame with the target data (The "Risk" column)
target_df = pd.DataFrame(df["Risk"])
target_df.head()

## Data Preparation

A logistic regression model will be trained using all the input features.

* `X` is the predictor variable vector with the values of all features.
* `Y` is the target variable vector with the risk result.

In [ ]:
# Encode the categorical features (One-hot encode)
features_enc = pd.get_dummies(features_df)
features_enc.head()

In [ ]:
# ignore DataConversionWarning messages
import warnings
from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings(action="ignore", category=DataConversionWarning)

In [ ]:
# Scale the features
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(features_enc)
scaler
print(scaler.mean_[:5])
print(scaler.scale_[:5])

In [ ]:
features_enc_scaled = scaler.transform(features_enc)
features_enc_scaled

In [ ]:
target_enc = pd.get_dummies(target_df)
target_enc.head()

In [ ]:
X = features_enc_scaled
Y = target_enc["Risk_bad"].values.reshape(-1)  # 0 = Good, 1 = Bad Risk
print(X[:5])
print(Y[:5])

In [ ]:
# Split the data in training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Machine Learning Model Creation

In [ ]:
bucket = "your_s3_bucket_name_here"
prefix = "german-credit-risk"

# Amazon SageMaker and related imports
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker import get_execution_role
import boto3  # AWS Python sdk

import os
import io
import json

# AWS IAM role
role = get_execution_role()

### Uploading Training Data to Amazon S3

In order to train your machine learning model using Amazon SageMaker, the training data should passed through an Amazon S3 Bucket formatted as a [protobuf recordIO format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#td-serialization).

The profobuf recordIO format, is a method to serialize structured data (similar to `JSON`), to allow different applications to communicate with each other or for storing data.

Using the profobuf recordIO format, allows you to take advantage of _Pipe mode_ when training the algorithms that support it. In _Pipe mode_, your training job streams data directly from Amazon S3. Streaming can provide faster start times for training jobs and better throughput.

The following code converts the training data as a Protocol Buffer, next the data is uploaded to the Amazon S3 bucket.

In [ ]:
# Encode the training data as Protocol Buffer


# Upload encoded training data to Amazon S3



#### Upload Test Data to Amazon S3

If you provide test data, the algorithm logs include the test score for the final model.

In [ ]:
# Encode the testing data as Protocol Buffer


# Upload encoded testing data to Amazon S3



### Training the Machine Learning Model

Once you have uploaded your data to Amazon S3, it's time to train the machine learning model. In this activity, you will use the Amazon SageMaker's [_linear learner algorithm_](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html) to run a linear regression prediction model.

You can learn more about the diferent Amazon SageMaker built-in algorithms [in this page](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html).

First, an intance of the linear learner algorithm is created.

In [ ]:
# Create an instance of the linear learner algorithm
from sagemaker.amazon.amazon_estimator import get_image_uri

container = 


Next, the estimator container is created in an AWS EC2 instance using a `ml.m4.xlarge`.

**Note:** This step might take a few minutes.

In [ ]:
# Start the Amazon SageMaker session
sess = 

# Create an instance of the linear learner estimator
linear = sagemaker.estimator.Estimator(
    container,
    role,
    train_instance_count=1,
    train_instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)

# Get the dimension of the feature-input vector
feature_dim = len(X[:1][0])


# Define linear learner hyperparameters
# Note how in this case we use: predictor_type='binary_classifier' # (credit risk: good or bad)
linear.set_hyperparameters(
    feature_dim=feature_dim, mini_batch_size=200, predictor_type="binary_classifier"
)

# Fitting the linear learner model with the training data
linear.fit({"train": s3_train_data, "test": s3_test_data})

### Deploying the Model to Make Predictions

In this section, the `linear-learner` model that was trained will be used to make predictions of credit risk. Deploy the model using a `ml.t2.medium` instance type.

**Note:** This step might take a few minutes.

In [ ]:
# An instance of the linear-learner predictor is created
linear_predictor = 

In [ ]:
# Linear predictor configurations
linear_predictor.content_type = "text/csv"
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [ ]:
# Making some predictions using the test data.
result = linear_predictor.predict(X_test)
y_predictions = np.array(
    [np.uint8(r["predicted_label"]) for r in result["predictions"]]
)
y_predictions

### Model Evaluation

To evaluate the model, we create a confusion matrix to compare the cases where each predicted value matches the expected test-value and when not.

In [ ]:
import pandas as pd

# Encode the predictios results as 0 = Good, 1 = Bad Risk
cat_test = np.array(["Good" if x == 0 else "Bad" for x in Y_test])
cat_pred = np.array(["Good" if x == 0 else "Bad" for x in y_predictions])

# Create the confusion matrix
pd.crosstab(cat_test, cat_pred, rownames=["actuals"], colnames=["predictions"])

The model is also evaluated using the `sklearn` metrics module. The following metrics are calculated:

* [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

* [`confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

* [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(f"accuracy_score: {accuracy_score(Y_test,y_predictions)}\n\n")
print(
    f"sklearn's confusion_matrix: \n\n{confusion_matrix(Y_test, y_predictions, labels=[0, 1])}\n\n"
)
print(
    f"classification_report: \n\n{classification_report(Y_test, y_predictions, target_names=['Good', 'Bad'])}\n\n"
)

Finally the end point is deleted to avoid additional AWS resources usage.

In [ ]:
# Delete Amazon SageMaker end-point

